In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

!pip install bayesian-optimization

from sklearn.model_selection import KFold,StratifiedKFold   # K-fold CV 
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import warnings
warnings.filterwarnings(action = 'ignore')

from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/201901-202003.csv')

# 결측치에 대한 고찰

In [ ]:
data.isna().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

In [ ]:
df=data.copy()

업종 관련 및 소비자 관련의 CCG 변수 2개 모두 많지는 않지만 어느정도 결측치를 가지고 있음.

In [ ]:
print('CARD_CCG,HOM_CCG 모두 NA의 행 개수:',data[(data['CARD_CCG_NM'].isna())&(data['HOM_CCG_NM'].isna())].shape[0])
print('CARD_CCG,HOM_CCG 모두 NA일 때, CARD_SIDO_NM:',data[(data['CARD_CCG_NM'].isna())&(data['HOM_CCG_NM'].isna())]['CARD_SIDO_NM'].unique())
print('CARD_CCG,HOM_CCG 모두 NA일 때, CARD_HOM_NM:',data[(data['CARD_CCG_NM'].isna())&(data['HOM_CCG_NM'].isna())]['HOM_SIDO_NM'].unique())

CARD_CCG,HOM_CCG 모두 NA의 행 개수: 7736
CARD_CCG,HOM_CCG 모두 NA일 때, CARD_SIDO_NM: ['세종']
CARD_CCG,HOM_CCG 모두 NA일 때, CARD_HOM_NM: ['세종']


우선 모든 CCG변수 NA 값인 경우 SIDO_NM이 둘다 '세종'으로 나타나 있다. 세종에는 어떤 지역들(CCG)가 있을까?

In [ ]:
print('CARD_SIDO_NM이 세종인 개수',data[data['CARD_SIDO_NM']=='세종'].shape[0])
print('세종의 결측치 개수',len(data[data['CARD_SIDO_NM']=='세종']['CARD_CCG_NM'].isna()))

CARD_SIDO_NM이 세종인 개수 87213
세종의 결측치 개수 87213


In [ ]:
print('HOM_SIDO_NM이 세종인 개수',data[data['HOM_SIDO_NM']=='세종'].shape[0])
print('세종의 결측치 개수',len(data[data['HOM_SIDO_NM']=='세종']['HOM_CCG_NM'].isna()))

HOM_SIDO_NM이 세종인 개수 147787
세종의 결측치 개수 147787


CARD_SIDO_NM이 세종인 경우 CARD_CCG_NM이,HOM_SIDO_NM이 세종인 경우 HOM_CCG_NM이 모두 결측치로만 구성되어 있다.
- 세종시는 소정면, 전의면, 전동면, 조치원읍, 연서면,연동면, 연기면,부강면, 금난면, 장군면으로 이루어져 있으나, 현재 데이터에는 세종시의 행정 구역이 따로 기재가 되어있지 않아있음.
> -> 그냥 CCG에도 '세종시'라고 넣어도 될 것으로 보인다.

In [ ]:
df.loc[df['CARD_SIDO_NM']=='세종','CARD_CCG_NM']='세종'
df.loc[df['HOM_SIDO_NM']=='세종','HOM_CCG_NM']='세종'

In [ ]:
df.isna().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

In [ ]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/train_df_noNAN.csv', encoding='utf-8-sig',index=False)